In [8]:
!pip install setuptools
!pip install setuptools
!pip install pandas

In [10]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, IntegerType
from pyspark.sql import functions as F
from collections import defaultdict
import networkx as nx

spark = SparkSession.builder.appName("CommunityDetectionAGM").getOrCreate()

edges_schema = StructType([
    StructField("id_1", IntegerType(), True),
    StructField("id_2", IntegerType(), True)
])
edges_df = spark.read.csv("musae_git_edges.csv", header=True, schema=edges_schema)

edges_list = [(row.id_1, row.id_2) for row in edges_df.collect()]

G = nx.Graph()
G.add_edges_from(edges_list)

from networkx.algorithms.community import greedy_modularity_communities
communities = list(greedy_modularity_communities(G))

modularity = nx.algorithms.community.quality.modularity(G, communities)

print("Communities Detected:")
for i, community in enumerate(communities):
    print(f"Community {i+1}: {sorted(community)}")

print(f"\nModularity Score: {modularity}")

communities_dict = defaultdict(list)
for i, community in enumerate(communities):
    for node in community:
        communities_dict[node] = i  

communities_df = spark.createDataFrame(
    [(node, community_id) for node, community_id in communities_dict.items()],
    schema=["node_id", "community_id"]
)

print("\nCommunity Assignments:")
communities_df.show()

spark.stop()

Communities Detected:
Community 1: [0, 2, 5, 6, 7, 10, 12, 16, 17, 18, 19, 21, 22, 23, 24, 28, 30, 39, 40, 42, 50, 53, 54, 55, 57, 58, 61, 63, 70, 73, 85, 87, 91, 92, 93, 95, 96, 97, 98, 100, 102, 119, 120, 122, 128, 134, 140, 141, 143, 144, 150, 154, 156, 160, 164, 175, 178, 181, 185, 186, 188, 190, 192, 194, 195, 196, 200, 209, 211, 216, 219, 221, 222, 227, 230, 231, 236, 237, 240, 241, 243, 244, 245, 250, 251, 252, 255, 256, 260, 261, 262, 264, 268, 275, 277, 278, 281, 284, 286, 287, 291, 292, 294, 295, 305, 306, 309, 311, 314, 316, 318, 319, 321, 323, 326, 328, 330, 332, 333, 335, 336, 339, 342, 346, 348, 349, 351, 352, 353, 354, 356, 357, 358, 360, 362, 371, 372, 378, 380, 381, 382, 388, 391, 393, 396, 397, 400, 402, 403, 407, 409, 412, 413, 416, 418, 419, 425, 426, 433, 437, 440, 443, 444, 447, 452, 453, 457, 458, 459, 460, 463, 466, 469, 471, 473, 477, 484, 487, 489, 491, 493, 495, 496, 497, 498, 500, 508, 509, 511, 515, 523, 524, 525, 531, 534, 541, 544, 545, 550, 553, 557, 565

Py4JJavaError: An error occurred while calling o138.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 2.0 failed 1 times, most recent failure: Lost task 0.0 in stage 2.0 (TID 2) (Akshat executor driver): org.apache.spark.SparkException: Python worker exited unexpectedly (crashed)
	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator$$anonfun$1.applyOrElse(PythonRunner.scala:612)
	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator$$anonfun$1.applyOrElse(PythonRunner.scala:594)
	at scala.runtime.AbstractPartialFunction.apply(AbstractPartialFunction.scala:38)
	at org.apache.spark.api.python.PythonRunner$$anon$3.read(PythonRunner.scala:789)
	at org.apache.spark.api.python.PythonRunner$$anon$3.read(PythonRunner.scala:766)
	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator.hasNext(PythonRunner.scala:525)
	at org.apache.spark.InterruptibleIterator.hasNext(InterruptibleIterator.scala:37)
	at scala.collect

In [12]:
print("\nCommunity Assignments:")
print("node_id    community_id")
print("-" * 25)
for node, community in sorted(communities_dict.items()):
    print(f"{node:<10} {community}")


Community Assignments:
node_id    community_id
-------------------------
0          0
1          1
2          0
3          2
4          5
5          0
6          0
7          0
8          1
9          2
10         0
11         1
12         0
13         3
14         1
15         1
16         0
17         0
18         0
19         0
20         1
21         0
22         0
23         0
24         0
25         1
26         1
27         1
28         0
29         1
30         0
31         1
32         2
33         2
34         2
35         1
36         1
37         2
38         1
39         0
40         0
41         1
42         0
43         1
44         1
45         35
46         3
47         2
48         2
49         2
50         0
51         3
52         3
53         0
54         0
55         0
56         2
57         0
58         0
59         2
60         236
61         0
62         1
63         0
64         2
65         1
66         2
67         1
68         1
69         3
70         0


In [11]:
print("\nGraph Information:")
print(f"Number of nodes: {G.number_of_nodes()}")
print(f"Number of edges: {G.number_of_edges()}")
print("\nGraph Density:", nx.density(G))
print("\nAverage clustering coefficient:", nx.average_clustering(G))
print("\nDegree Distribution:")
degrees = [d for n, d in G.degree()]
print(f"Average Degree: {sum(degrees)/len(degrees):.2f}")
print(f"Maximum Degree: {max(degrees)}")
print(f"Minimum Degree: {min(degrees)}")

print("\nFirst 10 edges in the graph:")
for edge in list(G.edges())[:10]:
    print(edge)



Graph Information:
Number of nodes: 37700
Number of edges: 289003

Graph Density: 0.0004066878203117068

Average clustering coefficient: 0.1675370448010758

Degree Distribution:
Average Degree: 15.33
Maximum Degree: 9458
Minimum Degree: 1

First 10 edges in the graph:
(0, 23977)
(23977, 27803)
(23977, 69)
(23977, 1966)
(23977, 2939)
(23977, 3147)
(23977, 4422)
(23977, 5631)
(23977, 5895)
(23977, 25679)
